![Image](./snapshot_strategy.png)

In [1]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
# p = p+r'\config'
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [17]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se
import re
import datetime

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [8]:
q_sql_2223 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-12-01' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [35]:
q_sql_2324 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from public_for_2324.posts a 
               , public_for_2324.postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2024-12-01' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [36]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2223)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [37]:
q_output_for_2223 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [38]:
q_output_for_2223['22_yn'] = np.where(pd.to_datetime(q_output_for_2223['creationdate']).dt.date < datetime.date(2022, 11, 30), 1, 0)

q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

In [39]:
qid_for_2223_22 = np.random.choice(list(q_output_for_2223_22['id']), size=54, replace=False)
qid_for_2223_23 = np.random.choice(list(q_output_for_2223_23['id']), size=54, replace=False)

In [40]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2324)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [41]:
q_output_for_2324 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [42]:
q_output_for_2324['22_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2022, 11, 30), 1, 0)
q_output_for_2324['23_yn'] = np.where((pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2023, 11, 30)) & (q_output_for_2324['22_yn'] ==0), 1, 0)
q_output_for_2324['24_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date >= datetime.date(2023, 11, 30), 1, 0)

print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].max())

# q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
# q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

2021-12-18 13:28:11.557000
2022-11-29 23:59:43.813000
2022-11-30 00:03:46.690000
2023-11-29 23:58:01.407000
2023-11-30 00:31:27.690000
2024-11-30 23:59:32.580000


In [27]:
q_output_copy = q_output[q_output['id'].isin(first_ann_q_id)].copy()

In [28]:
def chg_tag(code):
    st_pattern = r'<pre(?: class="[^"]*")?><code>'
    st_dst = "```python\n"
    code = re.sub(st_pattern, st_dst, code, count=0, flags=0)
    
    end_dst = "```"
    end_pattern =r'</code></pre>'
    code = re.sub(end_pattern, end_dst, code, count=0, flags=0)
    return code
                                                

In [29]:
q_output_copy['t_body'] = q_output_copy['body'].apply(chg_tag)

In [30]:
q_output_copy['clean_body'] = q_output_copy['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))

In [31]:
q_output_copy['clean_body'] = q_output_copy['clean_body'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
# .str.replace('";', "")

In [32]:
q_output_copy['question'] = """<Title>"""+q_output_copy['title'].map(str)+"""</Title>. <Question>"""+q_output_copy['clean_body'].map(str)+"""</Question> Let's think through the difficulty of question carefully, step by step. """


In [33]:
q_output_copy.head()

,id,title,body,t_body,clean_body,question
7173,77381677,Why does plotly.Mesh3d refuse to plot a 3D fun...,<p>I am attempting to plot the analytical t=0 ...,<p>I am attempting to plot the analytical t=0 ...,I am attempting to plot the analytical t=0 sol...,<Title>Why does plotly.Mesh3d refuse to plot a...
15036,77542349,How i can get correct result through f-strings,<pre><code>print(f&quot;{ {x ** 2 if (x**2) &l...,```python\nprint(f&quot;{ {x ** 2 if (x**2) &l...,"```python\nprint(f""{ {x ** 2 if (x**2) <= 10 e...",<Title>How i can get correct result through f-...
57314,76053816,create a new column named as group_ID in panda...,<p>create a new column named as <code>group_ID...,<p>create a new column named as <code>group_ID...,create a new column named as group_ID in panda...,<Title>create a new column named as group_ID i...
58394,76030780,Code producing incorrect numbers in Heat Excha...,<p>I am coding a heat exchange simulation. The...,<p>I am coding a heat exchange simulation. The...,I am coding a heat exchange simulation. The va...,<Title>Code producing incorrect numbers in Hea...
61611,76080898,Is there a way to convert an array in a string...,<p>I tried converting an array that is in a st...,<p>I tried converting an array that is in a st...,I tried converting an array that is in a strin...,<Title>Is there a way to convert an array in a...


In [35]:
q_output_copy[['id', 'question']].to_csv('./ICR_250408.csv')